
# Factoring 15 with Shor's Algorithm



## Introduction

The integer factorization problem [[1](#IntegerFactor)] is a famous problem in number theory: given a number $N$ which is composite, find its prime factors. The importance of the problem stems from the fact that no efficient (polynomial-time, in the number of bits needed to represent $N$) classical algorithm is known for it to this day, and much of modern day cryptography relies on this fact. In 1994, Peter Shor came up with an efficient _quantum_ algorithm for the problem [[2](#Shor94)] - providing one of the first concrete pieces of evidence for the power of quantum computers.

### Shor's Algorithm

Shor's algorithm consists of a classical part and a quantum subroutine. The steps of the algorithm for factoring an input number $N$, summarized from [[3](#ShorSteps)], are as follows:

1. Pick a random number $1 < a < N$ that is co-prime with $N$. Co-primality can be checked by computing the GCD (greatest common divisor) of $a$ and $N$ - if it is 1 then we have found a co-prime $a$, otherwise we have found a non-trivial factor of $N$ and we are done.
2. Find the period $r$ of the following function, using the quantum period finding algorithm (described in [[4](#PeriodFinding)]): $$f(x) = a^x \mod N$$
3. If $r$ is odd or $a^{r/2} = -1 \mod N$, return to step 1 (this event can be shown to happen with probability at most $1/2$).
4. Otherwise, $\gcd(a^{r/2} \pm 1, N)$ are both factors of $N$, and computing one of them yields the required result.

In this demo, we will factor the number $N=15$ using Shor's algorithm, by applying the quantum subroutine (step 2) with $a=7$. This particular $a$ is chosen since it is co-prime with 15 and satisfies the conditions of step 3, providing us with a high probability of finding a factor of $N$.


## Setup

In this demo, besides the `classiq` package, we'll use the following packages:

We also have the following imports necessary for programming the quantum subroutine:

In [ ]:
import collections
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np

from classiq import ControlState, Model, QReg
from classiq.builtin_functions import QFT, HGate, UnitaryGate, XGate
from classiq.execution import (
    IBMBackendPreferences,
    QuantumInstructionSet,
    QuantumProgram,
)

## Building the quantum period finding circuit

We begin by declaring the number of qubits in the upper (counting) register the quantum subroutine uses. In our case, $N = 15$, and according to the algorithm the upper register must contain $q = \log(Q)$ qubits for $Q$ such that $N^2 \le Q < 2N^2$, namely $225 < Q < 450$, and therefore $q = 8$:

In [ ]:
counting_qubits_amount = 8

We now define a function which returns the unitary matrix $U$ (in list form) used in the period finding, which is specific to $N=15$ and $a=7$. Note that this unitary implicitly determines the number of qubits of the lower (auxiliary) register of the quantum subroutine - in this case, 4 (this number does not need to be specified and is inferred automatically by the synthesis engine).

In [ ]:
def shor_u_pow_k_N15_a7(k: int = 1) -> List[List[complex]]:
    # fmt: off
    swap = np.array(
        [
            [1, 0, 0, 0],
            [0, 0, 1, 0],
            [0, 1, 0, 0],
            [0, 0, 0, 1]
        ],
        dtype=complex
    )
    # fmt: on
    swap32 = np.kron(np.identity(4), swap)
    swap21 = np.kron(np.kron(np.identity(2), swap), np.identity(2))
    swap10 = np.kron(swap, np.identity(4))
    x = np.array([[0, 1], [1, 0]])
    x_all = np.kron(np.kron(x, x), np.kron(x, x))
    u = x_all @ swap10 @ swap21 @ swap32
    u_pow_k = np.linalg.matrix_power(u, k)
    return u_pow_k.tolist()

We now begin building the Classiq model for the circuit, by initializing a `Model` object:

In [ ]:
model = Model()

At the first layer of the quantum circuit, we prepare the equal superposition state in the top (counting) register, and prepare the $|1\rangle$ state in the bottom (auxiliary) register:

In [ ]:
h_outputs = []
for i in range(counting_qubits_amount):
    h_outputs.append(model.HGate(params=HGate()))

qubit_1_output = model.XGate(params=XGate())

We now apply the second layer of the circuit, which consists of controlled $U^{2^i}$ gates. We wire the input of the gate number $i$ using the `create_cu_input` function, which simply wires each control qubit to the correct qubit from the counting register and the target qubits to the output of the previous gate's target qubits (the first controlled gate is a special case to which we connect the $|1\rangle$ state).
Also note the usage of the built-in `UnitaryGate` function, which allows creating a controlled version of an arbitrary unitary gate, specified by the `shor_u_pow_k_N15_a7` function we previously defined.

In [ ]:
def create_cu_input(idx: int) -> Dict[str, QReg]:
    in_wires = {f"CTRL_{idx}": h_outputs[idx]["TARGET"]}
    if idx == 0:
        in_wires["TARGET[0]"] = qubit_1_output["TARGET"]
    else:
        in_wires["TARGET"] = cu_outputs[idx - 1]["TARGET"]
    return in_wires


cu_outputs = []
for idx in range(counting_qubits_amount):
    in_wires = create_cu_input(idx)
    unitary_params = UnitaryGate(data=shor_u_pow_k_N15_a7(2**idx))
    control_states = [ControlState(num_ctrl_qubits=1, name=f"CTRL_{idx}")]
    cu_outputs.append(
        model.UnitaryGate(
            params=unitary_params, control_states=control_states, in_wires=in_wires
        )
    )

Lastly, we apply the inverse quantum fourier transform (QFT) on the counting register:

In [ ]:
qft_in = dict((f"OUT[{idx}]", out[f"CTRL_{idx}"]) for idx, out in enumerate(cu_outputs))
model.QFT(
    params=QFT(num_qubits=counting_qubits_amount), is_inverse=True, in_wires=qft_in
);

We now send the model to the synthesis engine, taking roughly 15 seconds:

In [ ]:
model.sample()
qmod = model.get_model()

In [ ]:
with open("shor.qmod", "w") as f:
    f.write(qmod)

In [ ]:
from classiq import synthesize

qprog = synthesize(qmod)

We can now view the circuit and its depth. Note that since $U^4 = I$ (identity), the Classiq synthesis engine recognizes that all the controlled-$U^{2^i}$ gates with $i \geq 2$ are identity and are thus empty, as evident in the interactive visualizer:

In [ ]:
from classiq import show

show(qprog)

## Executing the circuit

Now, we turn to executing the circuit above, using the simulator:

In [ ]:
from classiq import execute

results = execute(qprog).result()

In [ ]:
from classiq.execution import ExecutionDetails

res = results[0].value

Since we are interested only in the measurement results of the counting register (the `counting_qubits_amount = 8` LSBs of the circuit), we prepare a new histogram based on these qubits only:

In [ ]:
hist_counting_qubits = collections.defaultdict(int)
for key, value in res.counts.items():
    newkey = key[-counting_qubits_amount:]
    hist_counting_qubits[newkey] += value

Plotting the result:

In [ ]:
plt.bar(hist_counting_qubits.keys(), hist_counting_qubits.values())

Since we are interested only in the measurement results of the counting register (the `counting_qubits_amount = 8` LSBs of the circuit), we prepare a new histogram based on these qubits only:

We obtained 4 results $y$ from the circuit, each with probability roughly $1/4$: $0, 64, 128$ and $192$. Dividing by $Q = 256$ we obtain 4 reduced fractions: $0, 1/4, 1/2$ and $3/4$, with two of them having the correct period $r=4$ in the denominator. With this period, we can compute the factors of $N = 15$: $\gcd(a^{r/2} \pm 1, N) = \gcd(7^2 \pm 1, 15) = 3, 5$.

## References

<a id='IntegerFactor'>[1]</a>: [Integer Factorization (Wikipedia)](https://en.wikipedia.org/wiki/Integer_factorization)

<a id='Shor94'>[2]</a>: [Shor, Peter W. "Algorithms for quantum computation: discrete logarithms and factoring." Proceedings 35th annual symposium on foundations of computer science. Ieee, 1994.](https://ieeexplore.ieee.org/abstract/document/365700)

<a id='ShorSteps'>[3]</a>: [Shor's Algorithm Procedure (Wikipedia)](https://en.wikipedia.org/wiki/Shor%27s_algorithm#Procedure)

<a id='PeriodFinding'>[4]</a>: [Quantum Period Finding (Wikipedia)](https://en.wikipedia.org/wiki/Shor%27s_algorithm#Quantum_part:_period-finding_subroutine)
